<a href="https://colab.research.google.com/github/joseorlandomx/topicodeindustria1/blob/main/Proyecto_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Proyecto Final (Dashboard tema libre)**

José Orlando Salas Contreras (208743738)

[jose.salas4373@alumnos.udg.mx](mailto:jose.salas4373@alumnos.udg.mx)

Verónica Marisela Frías Peralta (208561486)

[veronica.frias6148@alumnos.udg.mx](mailto:veronica.frias6148@alumnos.udg.mx)

**Requisitos mínimos:**
* **Capa 1**: Un panel de controles que permita generar el **dataset origen** a analizar. Este dataset puede ser generado al momento partiendo de una formula e introduciendo los parametros a travez de los widgets en el panel (**ejemplo**: selección del tipo de trayectoria y sus parámetros en la práctica 4), o puede ser el resultado de filtrar un dataset mayor ya existente en memoria, en cuyo caso los widgets en el panel de control permitiran ajustar los limites de dichos filtros.
* **Capa 2**: Un panel en el cual se despliegue una **representación gráfica del dataset origen** (**ejemplo**: Trazo de la trayectoria en 3D en la práctica 4).
* **Capa 3**: Un panel de controles que permita elegir entre diferentes **metricas** a calcular a partir del dataset origen. Las métricas obtenidas deberán ser representadas también graficamente (**ejemplo**: Selección entre path length y mean squared displacement y su gráfica en la práctica 4).

**Revisión de la práctica:**
* El notebook con la práctica se entregará en un repositorio en **GitHub**. Será necesario demostrar por medio de commits el historial de versiones de su NoteBook.
* Los integrantes del equipo deberán explicar su código y deberán ser capaces de corregir errores introducidos a este.
* Uno de los integrantes del equipo presentará el dashboard durante la clase.

## **Modules**

In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import panel as pn
import panel.widgets as pnw
import plotly.express as px
from scipy.stats import wrapcauchy
from scipy.stats import levy_stable
from scipy.spatial import distance
from google.colab import drive
pn.extension('plotly')

In [4]:
url = '/content/conjunto_de_datos_gastotarjetas_enigh2022_ns.csv'
df = pd.read_csv(url)

## **Capa 1**

In [ ]:
# Capa 1: Panel de Controles para Generar el Dataset
def generate_dataset(tipo_trayectoria, parametro1, parametro2):
    # Lógica para generar el dataset según la fórmula o filtros
    # Utiliza los parámetros proporcionados

# Crear widgets para la Capa 1
tipo_trayectoria_widget = widgets.Dropdown(options=['Opción1', 'Opción2'], description='Tipo de Trayectoria:')
parametro1_widget = widgets.FloatSlider(min=0, max=10, step=0.1, value=5, description='Parámetro 1:')
parametro2_widget = widgets.FloatSlider(min=0, max=10, step=0.1, value=5, description='Parámetro 2:')

# Crear panel interactivo para la Capa 1
capa1_panel = interactive(generate_dataset, tipo_trayectoria=tipo_trayectoria_widget,
                           parametro1=parametro1_widget, parametro2=parametro2_widget)

## **Capa 2**

In [ ]:
# Capa 2: Panel para Visualizar Gráficamente el Dataset
def plot_dataset():
    # Lógica para visualizar gráficamente el dataset
    # Utiliza los datos generados o filtrados en la Capa 1
    fig = px.scatter_3d(df, x='...', y='...', z='...', color='...')
    fig.show()

# Crear panel para la Capa 2
capa2_panel = interactive(plot_dataset)

## **Capa 3**

In [ ]:
# Capa 3: Panel de Controles para Elegir Métricas
def calculate_metrics(metrica):
    # Lógica para calcular y visualizar métricas
    # Utiliza los datos generados o filtrados en la Capa 1
    fig = px.bar(df, x='...', y=metrica, color='...')
    fig.show()

# Crear widgets para la Capa 3
metrica_widget = widgets.Dropdown(options=['path length', 'mean squared displacement'], description='Seleccione Métrica:')

# Crear panel interactivo para la Capa 3
capa3_panel = interactive(calculate_metrics, metrica=metrica_widget)

In [2]:
# Mostrar los paneles
display(capa1_panel)
display(capa2_panel)
display(capa3_panel)

IndentationError: expected an indented block after function definition on line 2 (<ipython-input-2-80c0baadcd26>, line 7)